In [1]:
import pandas as pd 
import numpy as np 
import seaborn as sns 
import matplotlib.pyplot as plt 
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences
from nltk.tokenize import word_tokenize
import tensorflow as tf 
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Embedding,LSTM,Bidirectional,Dropout,BatchNormalization
from keras.optimizers.legacy import Adam 
from keras.callbacks import EarlyStopping
from keras.regularizers import l1,l2

/Users/ripeshghimire/coding/PoetryGeneration/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
with open("/Users/ripeshghimire/coding/PoetryGeneration/data/poem 2.txt",'r') as file:
    corpus = file.readlines() 

Finding the longest element and the longest sentence in the corpus

In [3]:
max_len = 0
longest_list = None
for current_list in corpus :
    length_word = len(current_list)
    if length_word > max_len:
        max_len = length_word
        longest_list = current_list

max_len

81

Peforming preprocessing in the dataset 

In [22]:
def preprocess(corpus):
#  tokenizer = Tokenizer()
#  tokenizer.fit_on_texts(corpus)
 preprocess_corpus = []
 for sentence in corpus:
    sentence = sentence.replace(',','')
    tokens = word_tokenize(sentence) 
    preprocess_sent = [word.lower() for word in tokens]  
    preprocess_corpus.append(preprocess_sent)
 return preprocess_corpus

In [23]:
preprocess_corpus = preprocess(corpus)
#maxlen for val data 

Splitting the data into train test 

In [24]:
train_data, temp_data = train_test_split(preprocess_corpus, train_size=0.8, random_state=10)
test_data,val_data= train_test_split(temp_data, test_size=0.8, random_state=10)

Encoding the data using Tokenizer and pad_sequence

In [25]:
tokenizer = Tokenizer()
def encode_poem(data,max_len = 81):
    encoded_train_data = []
    for sentences in data:
        tokenizer.fit_on_texts([sentences])
        sequence = tokenizer.texts_to_sequences([sentences])[0]
        for i in range(1, len(sequence)):
            n_gram_sequence = sequence[:i+1]
            encoded_train_data.append(n_gram_sequence)
    padded_encoded_train_data = pad_sequences(encoded_train_data, maxlen=max_len, padding='pre')
    return np.array(padded_encoded_train_data)

In [26]:
encoded_train = encode_poem(train_data)
encoded_val = encode_poem(val_data)


Defining Features and labels in the data

In [33]:
total_words = len(tokenizer.word_index)+1
def prepare_data(training_data,total_words =total_words):
    xs, labels = training_data[:,:-1],training_data[:,-1]
    ys = tf.keras.utils.to_categorical(labels, num_classes=total_words)
    return xs,ys

In [34]:
X_train,y_train = prepare_data(encoded_train)
X_train_val,y_train_val = prepare_data(encoded_val)

Training the model

In [37]:
def create_model(learning_rate=0.001, lstm_units=100, epochs=100, batch_size=16):
    model = Sequential()
    model.add(Embedding(total_words, 240, input_length=max_len-1))
    model.add(Bidirectional(LSTM(lstm_units,kernel_regularizer=l2(l2=0.01))))
    model.add(BatchNormalization())
    model.add(Dropout(0.4))
    
    model.add(Dense(total_words, activation='softmax'))
    
    adam = Adam(learning_rate)
    model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])
    
    earlystop = EarlyStopping(monitor='val_loss', min_delta=0, patience=5, verbose=0, mode='auto')
    
    history = model.fit(X_train, y_train, epochs=epochs, verbose=1, batch_size=batch_size,
                        callbacks=[earlystop],validation_data=(X_train_val,y_train_val))
    
    return model, history


In [38]:
trained_model,trained_history = create_model()

Epoch 1/100
857/857 [==============================] - 39s 44ms/step - loss: 7.1257 - accuracy: 0.0584 - val_loss: 6.8639 - val_accuracy: 0.0618
Epoch 2/100
857/857 [==============================] - 37s 43ms/step - loss: 6.4765 - accuracy: 0.0739 - val_loss: 8.4628 - val_accuracy: 0.0371
Epoch 3/100
857/857 [==============================] - 37s 44ms/step - loss: 6.3332 - accuracy: 0.0854 - val_loss: 7.3651 - val_accuracy: 0.0575
Epoch 4/100
857/857 [==============================] - 39s 46ms/step - loss: 6.1473 - accuracy: 0.0970 - val_loss: 7.5651 - val_accuracy: 0.0778
Epoch 5/100
857/857 [==============================] - 40s 46ms/step - loss: 5.9149 - accuracy: 0.1120 - val_loss: 8.6752 - val_accuracy: 0.0375
Epoch 6/100
857/857 [==============================] - 40s 47ms/step - loss: 5.6476 - accuracy: 0.1293 - val_loss: 9.4746 - val_accuracy: 0.0458


In [ ]:
trained_model.save('model.h5')

In [ ]:
from keras.models import load_model
model = load_model('model.h5')

1/1 [==============================] - 0s 19ms/step
How are you ] take     i            bowing madan to  ?    meat       on          left      
